# Функция для численного интегрирования площади под кривой

In [3]:
def integrate(f, a, b, *, n_iter=1000):
    """
    Численное интегрирование площади под кривой методом трапеций.
    
    Параметры:
    f : функция одной переменной
    a : нижний предел интегрирования
    b : верхний предел интегрирования
    n_iter : количество итераций (по умолчанию 1000)
    
    Возвращает:
    float: приближенное значение интеграла
    """
    h = (b - a) / n_iter
    result = 0.5 * (f(a) + f(b))
    
    for i in range(1, n_iter):
        x = a + i * h
        result += f(x)
    
    result *= h
    
    return result

# Пример использования:
import math

# Интегрирование sin(x) от 0 до pi
result = integrate(math.sin, 0, math.pi, n_iter=1000)
print(f"Интеграл sin(x) от 0 до pi: {result}")
print(f"Точное значение: 2.0")

Интеграл sin(x) от 0 до pi: 1.9999983550656624
Точное значение: 2.0


# Оценка скорости выполнения интегрирования 

In [4]:
import math
import timeit

def integrate(f, a, b, *, n_iter=1000):
    h = (b - a) / n_iter
    result = 0.5 * (f(a) + f(b))
    for i in range(1, n_iter):
        x = a + i * h
        result += f(x)
    return result * h

# Оценка времени выполнения для разных n_iter
for n in [10**4, 10**5, 10**6]:
    time = timeit.timeit(
        f"integrate(math.sin, 0, math.pi, n_iter={n})",
        globals=globals(),
        number=100
    )
    print(f"n_iter = {n}: {time:.4f} секунд (среднее из 100 запусков)")

# Вычисление и вывод результатов интегрирования для проверки
for n in [10**4, 10**5, 10**6]:
    result = integrate(math.sin, 0, math.pi, n_iter=n)
    print(f"n_iter = {n}: результат = {result:.6f}")

print(f"Точное значение: 2.000000")

n_iter = 10000: 0.1486 секунд (среднее из 100 запусков)
n_iter = 100000: 1.4505 секунд (среднее из 100 запусков)
n_iter = 1000000: 14.4238 секунд (среднее из 100 запусков)
n_iter = 10000: результат = 2.000000
n_iter = 100000: результат = 2.000000
n_iter = 1000000: результат = 2.000000
Точное значение: 2.000000


# Многопоточное интегрирование

In [5]:
import math
import threading
import timeit

def integrate_part(f, a, b, n, result, lock):
    h = (b - a) / n
    local_sum = 0
    for i in range(n):
        x = a + i * h
        local_sum += f(x) * h
    with lock:
        result[0] += local_sum

def integrate_threaded(f, a, b, n_iter, n_threads):
    result = [0]
    lock = threading.Lock()
    threads = []
    chunk_size = n_iter // n_threads
    
    for i in range(n_threads):
        start = a + (b - a) * i / n_threads
        end = a + (b - a) * (i + 1) / n_threads
        thread = threading.Thread(target=integrate_part, args=(f, start, end, chunk_size, result, lock))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()
    
    return result[0]

def integrate_sequential(f, a, b, n_iter):
    h = (b - a) / n_iter
    result = 0
    for i in range(n_iter):
        x = a + i * h
        result += f(x) * h
    return result

# Параметры интегрирования
f = math.sin
a, b = 0, math.pi
n_iter = 10**6
n_threads = 4

# Измерение времени для многопоточного решения
time_threaded = timeit.timeit(
    lambda: integrate_threaded(f, a, b, n_iter, n_threads),
    number=10
) / 10

# Измерение времени для последовательного решения
time_sequential = timeit.timeit(
    lambda: integrate_sequential(f, a, b, n_iter),
    number=10
) / 10

# Вычисление результатов для проверки точности
result_threaded = integrate_threaded(f, a, b, n_iter, n_threads)
result_sequential = integrate_sequential(f, a, b, n_iter)

print(f"Многопоточное решение ({n_threads} потоков):")
print(f"Время: {time_threaded:.6f} секунд")
print(f"Результат: {result_threaded:.6f}")

print(f"\nПоследовательное решение:")
print(f"Время: {time_sequential:.6f} секунд")
print(f"Результат: {result_sequential:.6f}")

print(f"\nТочное значение: 2.000000")

print(f"\nУскорение: {time_sequential / time_threaded:.2f}x")

Многопоточное решение (4 потоков):
Время: 0.118943 секунд
Результат: 2.000000

Последовательное решение:
Время: 0.148726 секунд
Результат: 2.000000

Точное значение: 2.000000

Ускорение: 1.25x
